In [2]:
import sys, os, json, pickle, gzip, copy
sys.path.append('..')
#from src.data import BaseDataset, ConditionalData
import numpy as np
import gzip, pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
from src.utils import ActiveElements, MetalElements, Element
from src.utils import composit_parser

# ELMD

In [26]:
screened_precursor[0]

{'id': -1,
 'label': 0,
 'count': 0,
 'count_rxn': 0,
 'year': 0,
 'precursor_str': 'Unknown',
 'precursor_comp': {}}

In [27]:
with gzip.open('../data/unique_target.pkl.gz','rb') as f:
    unique_target = pickle.load(f)
with gzip.open('../data/screened_precursor.pkl.gz','rb') as f:
    screened_precursor = pickle.load(f)
num_eles = np.zeros((len(ActiveElements), 2))

for data in unique_target:
    for ele, frac in data['target_comp'].items():
        i = ActiveElements.index(ele)
        num_eles[i, 0] += data['count']
for data in screened_precursor.values():
    for ele, frac in data['precursor_comp'].items():
        i = ActiveElements.index(ele)
        num_eles[i, 1] += data['count']

In [28]:
elmd = {}
for fn in os.listdir('../src/elmd'):
    if not fn.endswith('json'): continue
    elmd[fn.split('.')[0]] = json.load(open(f'../src/elmd/{fn}'))

for fn, data in sorted(elmd.items(), key=lambda x: x[0]):
    dtype = []
    dlen = []
#    ele_not_in_table = []
    for ele, feat in data.items():
        dtype.append(str(type(feat)).split()[1].replace('>','').replace('\'',''))
        if isinstance(feat, (int, float)):
            dlen.append(1)
        else:
            dlen.append(len(feat))
#    for ele in elmd['atomic'].keys():
#        if ele not in data.keys() and ele not in 'DTUue':
#            ele_not_in_table.append((ele, Element(ele).number))
    ele_not_in_data = {}
    for i, ele in enumerate(ActiveElements):
        if ele not in data.keys() and num_eles[i].sum() != 0:
            ele_not_in_data[ele] = num_eles[i, :]
    print(fn, dtype[0], dlen[0], len(dtype))
#    print(ele_not_in_table)
    if len(ele_not_in_data) != 0:
        print(ele_not_in_data)
#    print(ele_not_in_table)
#    print()
    

atomic int 1 121
cgcnn list 91 100
elemnet list 135 98
jarvis list 438 82
{'Eu': array([1633., 1619.]), 'Np': array([11.,  8.]), 'Pu': array([26., 24.]), 'Am': array([4., 0.]), 'Cm': array([1., 0.])}
jarvis_sc list 438 82
{'Eu': array([1633., 1619.]), 'Np': array([11.,  8.]), 'Pu': array([26., 24.]), 'Am': array([4., 0.]), 'Cm': array([1., 0.])}
magpie list 21 97
magpie_sc list 22 97
mat2vec list 199 118
matscholar list 200 103
megnet16 list 16 95
{'Am': array([4., 0.]), 'Cm': array([1., 0.])}
mendeleev int 1 121
mod_petti int 1 121
oliynyk list 44 85
{'Np': array([11.,  8.]), 'Pu': array([26., 24.]), 'Am': array([4., 0.]), 'Cm': array([1., 0.])}
oliynyk_sc list 44 85
{'Np': array([11.,  8.]), 'Pu': array([26., 24.]), 'Am': array([4., 0.]), 'Cm': array([1., 0.])}
petti int 1 121
random_200 list 200 82
{'Eu': array([1633., 1619.]), 'Np': array([11.,  8.]), 'Pu': array([26., 24.]), 'Am': array([4., 0.]), 'Cm': array([1., 0.])}


In [21]:
len(ActiveElements)

97

# Composition featurization functions

In [114]:
with gzip.open('../data/unique_ligand.pkl.gz','rb') as f:
    unique_ligand = pickle.load(f)
with gzip.open('../data/unique_precursor.pkl.gz','rb') as f:
    unique_precursor = pickle.load(f)
with gzip.open('../data/unique_target.pkl.gz','rb') as f:
    unique_target = pickle.load(f)

In [115]:
from src.feature import composition_to_feature

composition_to_feature({'Li':0.2, 'Co':0.2, 'O':0.6}, feature_type='magpie')

array([[6.4000000e+01, 2.2774479e+01, 4.7721799e+02, 1.1600000e+01,
        2.4000001e+00, 9.0400002e+01, 2.6359999e+00, 1.8000000e+00,
        2.4000001e+00, 1.4000000e+00, 0.0000000e+00, 5.5999999e+00,
        2.0000000e-01, 1.2000000e+00, 6.0000002e-01, 0.0000000e+00,
        2.0000000e+00, 1.0830667e+01, 0.0000000e+00, 3.0969423e-01,
        9.1800003e+01]], dtype=float32)

In [15]:
fgs = {}
mask = np.ones(len(unique_precursor), dtype=bool)
for ele in MetalElements:
    precs = []
    for i, prec in enumerate(unique_precursor):
        if ele not in prec['precursor_comp'].keys(): continue
        precs.append([len(prec['years']), prec['precursor_comp']])
        mask[i] = False
    precs = sorted(precs, key=lambda x: x[0], reverse=True)
    precs = sorted(precs, key=lambda x: len(list(x[1].keys())))
    precs = sorted(precs, key=lambda x: tuple(list(x[1].keys())))
    for prec in precs:
        n = np.min(list(prec[1].values()))
        p = '   '.join([f'{e:2s}:{f/n:.2f}' for e, f in prec[1].items()])
        print('{:5d} : {}'.format(prec[0], p))
    if len(prec) != 0:
        print()

precs = []
for j in np.where(mask)[0]:
    prec = unique_precursor[j]
    precs.append([len(prec['years']), prec['precursor_comp']])
            
precs = sorted(precs, key=lambda x: x[0], reverse=True)
precs = sorted(precs, key=lambda x: len(list(x[1].keys())))
precs = sorted(precs, key=lambda x: tuple(list(x[1].keys())))
for prec in precs:
    n = np.min(list(prec[1].values()))
    p = '   '.join([f'{e:2s}:{f/n:.2f}' for e, f in prec[1].items()])
    print('{:5d} : {}'.format(prec[0], p))


   90 : Li:1.00
   22 : Li:1.00   Br:1.00
 3589 : Li:2.00   C :1.00   O :3.00
   57 : Li:1.00   C :1.00   O :3.00
    1 : Li:1.00   C :1.00   O :2.00
   28 : Li:1.00   Cl:1.00
  129 : Li:1.00   F :1.00
   22 : Li:1.00   H :1.00
  124 : Li:1.00   H :3.00   C :2.00   O :2.00
   21 : Li:1.00   H :2.00   N :1.00
  706 : Li:1.00   H :1.00   O :1.00
    1 : Li:1.00   H :2.00   S :1.00   N :1.00   O :3.00
    1 : Li:2.00   H :2.00   S :1.00   O :5.00
    1 : Li:1.00   I :1.00
   24 : Li:3.00   N :1.00
  138 : Li:1.00   N :1.00   O :3.00
  130 : Li:2.00   O :1.00
   26 : Li:1.00   O :1.00
    2 : Li:3.00   P :1.00
    1 : Li:1.00   P :2.33
   78 : Li:1.00   P :1.00   H :2.00   O :4.00
   47 : Li:3.00   P :1.00   O :4.00
    4 : Li:1.67   P :1.00   O :3.33
    4 : Li:1.40   P :1.00   O :3.20
    3 : Li:2.00   P :1.00   O :3.50
    2 : Li:1.00   P :1.00   O :3.00
    1 : Li:3.00   P :1.00   S :4.00
    1 : Li:6.00   P :1.00   S :5.00   Br:1.00
    1 : Li:4.00   P :1.00   Se:2.50   S :2.00
    1

In [18]:
fgs = {}
for prec in unique_precursor:
    n = len(prec['years'])
    metals = {}
    non_metals = {}
    for e, f in prec['precursor_comp'].items():
        if e in MetalElements:
            metals.update({e:f})
        else:
            non_metals.update({e:f})
    k_m = tuple(sorted(list(metals.keys()), key=lambda x: Element(x).number))
    k_fg = tuple(sorted(list(non_metals.keys()), key=lambda x: Element(x).number))
    if k_fg not in fgs.keys():
        fgs[k_fg] = {'count':0, 'metals':{}}
    if k_m not in fgs[k_fg]['metals']:
        fgs[k_fg]['metals'][k_m] = {
            'count':0, 'case':[]
        }
    fgs[k_fg]['count'] += n
    fgs[k_fg]['metals'][k_m]['count'] += n
    fgs[k_fg]['metals'][k_m]['case'].append([prec['precursor_comp'], n])

In [31]:
fgs = {k:v for k,v in sorted(fgs.items(), key=lambda x: len(x[0]))}
fgs = {k:v for k,v in sorted(fgs.items(), key=lambda x: x[0])}
fgs = {k:v for k,v in sorted(fgs.items(), key=lambda x: x[1]['count'])}
for k, v in fgs.items():
    m = list(v['metals'].keys())
    print(k, v['count'], f'{len(m)}/{len(MetalElements)}')
    print(m)
    print()

('C', 'N', 'S') 1 1/87
[()]

('H', 'C', 'N') 1 1/87
[()]

('H', 'C', 'N', 'Br') 1 1/87
[('Pb',)]

('H', 'O', 'Cl') 1 1/87
[('Sr',)]

('H', 'O', 'Se') 1 1/87
[()]

('H', 'S') 1 1/87
[()]

('O', 'F') 1 1/87
[('Tm',)]

('O', 'I') 1 1/87
[('Bi',)]

('O', 'P', 'Cl') 1 1/87
[('Ca',)]

('P', 'S', 'Br') 1 1/87
[('Li',)]

('S', 'Se') 1 1/87
[('Sn',)]

('H', 'C') 2 1/87
[()]

('H', 'C', 'N', 'Cl') 2 2/87
[(), ('Pb',)]

('H', 'C', 'N', 'I') 2 2/87
[(), ('Pb',)]

('H', 'C', 'N', 'O') 2 1/87
[()]

('H', 'C', 'N', 'O', 'P') 2 1/87
[('Fe',)]

('H', 'F') 2 1/87
[('Si',)]

('C', 'S') 3 2/87
[('Nb',), ('W',)]

('H', 'C', 'N', 'S') 3 1/87
[()]

('H', 'Cl') 3 2/87
[(), ('Au',)]

('N', 'P') 3 1/87
[()]

('O', 'F', 'P') 3 1/87
[('Na',)]

('P', 'S', 'Se') 3 2/87
[('Li',), ('Mg',)]

('H', 'C', 'S', 'Br') 4 1/87
[()]

('H', 'C', 'S', 'Cl') 4 1/87
[()]

('O', 'Br') 5 2/87
[('Pb',), ('Bi',)]

('H', 'C', 'S', 'I') 6 1/87
[()]

('H', 'N', 'Br') 6 1/87
[()]

('H', 'O', 'S') 14 3/87
[(), ('Ca',), ('Li',)]

('C', 'N'

# Data object test

In [32]:
def fnc1(x):
    return (x + 1) ** 3
def fnc2(x):
    return (x / 2 + 1) ** 0.5
def fnc3(x):
    return (x - 1) ** 2 / 2
def fnc4(x):
    return (x + 2) ** 2

def cfn1(n):
    d1 = []
    d2 = []
    d3 = []
    d4 = []
    for i in range(n):
        d1.append(fnc1(i))
        d2.append(fnc2(i))
        d3.append(fnc3(i))
        d4.append(fnc4(i))
    return d1, d2, d3, d4

def cfn2(n):
    d1 = []
    d2 = []
    d3 = []
    d4 = []
    for i in range(n):
        d1.append(fnc1(i))
    for i in range(n):
        d2.append(fnc2(i))
    for i in range(n):
        d3.append(fnc3(i))
    for i in range(n):
        d4.append(fnc4(i))
    return d1, d2, d3, d4

%timeit cfn1(500)
%timeit cfn2(500)

279 µs ± 552 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
283 µs ± 424 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
import sys
sys.path.append('..')
import numpy as np
import torch, os, gzip, pickle
from src.data import ReactionData
from typing import Dict, List

with gzip.open('../data/screened_conditional_reaction.pkl.gz','rb') as f:
    cond_rxn = pickle.load(f)
with gzip.open('../data/screened_unique_reaction.pkl.gz','rb') as f:
    uniq_rxn = pickle.load(f)

IndentationError: expected an indented block (data.py, line 441)

In [96]:
cnt = 0
for i, rxn in enumerate(uniq_rxn):
    eles = {}
    for prec in rxn['precursor_comp']:
        metal_ele = tuple([e for e in prec.keys() if e in MetalElements])
        if metal_ele not in eles.keys():
            eles[metal_ele] = 0
        eles[metal_ele] += 1
#    if np.max(list(eles.values())) > 2:
#        print(i, eles)
    if tuple([]) in eles:
        cnt += 1
        print(i, eles)
print(cnt, len(uniq_rxn))



77 {('Ba',): 1, ('In',): 1, (): 1}
103 {(): 1, ('Sc',): 1, ('Li',): 1}
104 {(): 1, ('Sc',): 1, ('Li',): 1}
125 {(): 1, ('Sc',): 1, ('Zr',): 1, ('Li',): 1}
171 {('Ba',): 1, ('Cu',): 1, (): 1}
183 {('Zn',): 1, ('Ga',): 1, (): 1}
184 {('Zn',): 1, ('Ga',): 1, (): 1}
187 {(): 1, ('Na',): 1}
188 {(): 1, ('Na',): 1}
189 {('Na',): 1, (): 1}
190 {('Na',): 1, (): 1}
191 {('Na',): 1, (): 1}
192 {(): 1, ('Na',): 1}
198 {('Eu',): 1, (): 1}
201 {('Si',): 1, ('Mo',): 1, (): 1}
225 {(): 1, ('Ti',): 1, ('Si',): 1}
226 {(): 1, ('Ti',): 1, ('Si',): 1}
229 {('Ba',): 1, ('Co',): 1, (): 1}
230 {('Ba',): 1, ('Ni',): 1, (): 1}
243 {('V',): 1, ('Ba',): 1, (): 1}
244 {('V',): 1, ('Ba',): 1, (): 1, ('K',): 1}
245 {('V',): 1, ('Ba',): 1, (): 1, ('K',): 1}
246 {('V',): 1, ('Ba',): 1, (): 1, ('K',): 1}
247 {('V',): 1, ('Ba',): 1, (): 1, ('K',): 1}
255 {(): 1, ('Bi',): 1, ('Te',): 1}
280 {('Ir',): 1, (): 1, ('Cu',): 1}
281 {('Ir',): 1, ('Cu',): 1, (): 1}
299 {(): 1, ('Y',): 1}
521 {('Li',): 1, (): 1, ('Zr',): 1}
522

In [97]:
uniq_rxn[77]

{'id': 87,
 'count': 1,
 'year': 2010,
 'target_comp': {'Ba': 0.22222222, 'In': 0.22222222, 'O': 0.5555556},
 'precursor_comp': [{'Ba': 0.2, 'C': 0.2, 'O': 0.6},
  {'In': 0.4, 'O': 0.6},
  {'P': 0.083333336, 'H': 0.5, 'N': 0.083333336, 'O': 0.33333334}],
 'dois': ['10.1039/c0cc00063a'],
 'heat_temp': [{'mean': None, 'median': None, 'raw': [1273.15, 1573.15]}],
 'heat_time': [{'mean': None, 'median': None, 'raw': [10.0, 12.0, 50.0]}]}

In [89]:
uniq_rxn[29022 ]

{'id': 30178,
 'count': 1,
 'year': 2019,
 'target_comp': {'Na': 0.0023809525,
  'Sr': 0.23333333,
  'Eu': 0.0023809525,
  'P': 0.14285715,
  'O': 0.5714286,
  'F': 0.04761905},
 'precursor_comp': [{'Eu': 0.4, 'O': 0.6},
  {'Sr': 0.2, 'C': 0.2, 'O': 0.6},
  {'Na': 0.5, 'F': 0.5},
  {'H': 0.625, 'N': 0.125, 'F': 0.25},
  {'P': 0.083333336, 'H': 0.5, 'N': 0.083333336, 'O': 0.33333334}],
 'dois': ['10.1016/j.jallcom.2019.151906'],
 'heat_temp': [{'mean': None,
   'median': None,
   'raw': [573.0, 873.0, 1073.0, 573.0, 873.0, 1073.0]}],
 'heat_time': [{'mean': None, 'median': None, 'raw': [12.0, 12.0]}]}

In [91]:
uniq_rxn[7168 ]

{'id': 7555,
 'count': 1,
 'year': 2006,
 'target_comp': {'Ca': 0.21428572,
  'Nd': 0.023809524,
  'Si': 0.023809524,
  'P': 0.11904762,
  'O': 0.5714286,
  'F': 0.04761905},
 'precursor_comp': [{'Si': 0.33333334, 'O': 0.6666667},
  {'Ca': 0.18181819, 'P': 0.18181819, 'O': 0.6363636},
  {'Ca': 0.5, 'O': 0.5},
  {'Nd': 0.4, 'O': 0.6},
  {'Ca': 0.33333334, 'F': 0.6666667}],
 'dois': ['10.1016/j.jnucmat.2006.02.058'],
 'heat_temp': [{'mean': None, 'median': None, 'raw': [1673.0]}],
 'heat_time': [{'mean': None, 'median': None, 'raw': [6.0]}]}

In [93]:
uniq_rxn[4]

{'id': 4,
 'count': 102,
 'year': 1988,
 'target_comp': {'Ba': 0.03125, 'Fe': 0.375, 'O': 0.59375},
 'precursor_comp': [{'Ba': 0.2, 'C': 0.2, 'O': 0.6}, {'Fe': 0.4, 'O': 0.6}],
 'dois': ['10.1007/bf00551283',
  '10.1016/s0925-8388(98)00766-x',
  '10.1016/s0025-5408(00)00382-2',
  '10.1016/s0025-5408(01)00751-6',
  '10.1016/s0304-8853(01)00288-8',
  '10.1016/s0304-8853(02)01387-2',
  '10.1016/j.jssc.2003.10.021',
  '10.1016/j.matlet.2004.05.017',
  '10.1016/s0304-8853(03)00559-6',
  '10.1016/s0304-8853(03)00586-9',
  '10.1016/j.jmmm.2004.07.033',
  '10.1016/j.matlet.2004.09.012',
  '10.1016/j.matlet.2005.01.072',
  '10.1016/j.powtec.2005.05.003',
  '10.1063/1.2058220',
  '10.1103/physrevb.72.104420',
  '10.1007/s10853-006-0921-y',
  '10.1016/j.jaap.2006.08.005',
  '10.1016/j.jmmm.2006.07.032',
  '10.1016/j.jmmm.2006.08.003',
  '10.1016/j.jmmm.2007.04.012',
  '10.1016/j.tsf.2007.03.013',
  '10.1007/s10854-007-9411-5',
  '10.1016/j.matlet.2007.09.073',
  '10.1002/pssa.200925104',
  '10.10

In [84]:
data = uniq_rxn[0]
rxn_data = TestData(data = data,
                        feat_type = 'composit',
                        target_comp = data['target_comp'],
#                        precursor_comps = data['precursor_comp'],
                        target_feat_by_fraction = True,
                        metal_feat_by_fraction = False,
                        precursor_feat_by_fraction = True,
#                        conditions = ['heat_temp'],
#                        condition_values = [data['target_comp']['heat_temp']['median']],
                        labels = 1,
                        weights = None,
)

0 1 {'Y': 0.0020005717} {'Al': 0.054872826}
0 2 {'Y': 0.0020005717} {'Si': 0.37153473}
0 3 {'Y': 0.0020005717} {}
N 0.5144327
O 0.057159193
1 0 {'Al': 0.054872826} {'Y': 0.0020005717}
1 2 {'Al': 0.054872826} {'Si': 0.37153473}
1 3 {'Al': 0.054872826} {}
N 0.5144327
O 0.057159193
2 0 {'Si': 0.37153473} {'Y': 0.0020005717}
2 1 {'Si': 0.37153473} {'Al': 0.054872826}
2 3 {'Si': 0.37153473} {}
N 0.5144327
O 0.057159193
3 0 {} {'Y': 0.0020005717}
N 0.5144327
O 0.057159193
3 1 {} {'Al': 0.054872826}
N 0.5144327
O 0.057159193
3 2 {} {'Si': 0.37153473}
N 0.5144327
O 0.057159193


In [27]:
rxn_data.metal_feat
rxn_data.target_comp
rxn_data.edge_feat

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5938, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.3750, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0312, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.

# Dataset test

In [1]:
import sys
sys.path.append('..')
import numpy as np
import torch, os
from src.data import CompositionDataset, ReactionDataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

## Dataset functions

In [14]:
ds = ReactionDataset()
ds.from_file('../data/screened_conditional_reaction.pkl.gz', heat_temp_key=('heat_temp', 'median'))

TypeError: attribute name must be string, not 'list'

In [15]:
ReactionData

NameError: name 'ReactionData' is not defined

In [9]:
ds[0].heat_temp

AttributeError: 'ReactionData' object has no attribute 'heat_temp'

## Model compatibility

In [2]:
years = np.array([d.year for d in DS])
DS.to('cuda')
train_idx = np.where(years < 2017)[0]
valid_idx = np.where((years > 2016) & (years < 2019))[0]
test_idx = np.where(years > 2018)[0]

train_dl = DataLoader(DS, batch_size=256, sampler=SubsetRandomSampler(train_idx), collate_fn=DS.cfn)
valid_dl = DataLoader(DS, batch_size=2048, sampler=valid_idx, collate_fn=DS.cfn)
test_dl = DataLoader(DS, batch_size=2048, sampler=valid_idx, collate_fn=DS.cfn)

model = DNNCVAE(DS.num_ligand_feat, 16, DS.num_target_feat + DS.num_metal_feat,
                encoder_hidden_dim=128, encoder_hidden_layers=4,
                decoder_hidden_dim=128, decoder_hidden_layers=4)

tr = VAETrainer(model, 1e-5)

path = '/home/jhyang/WORKSPACES/MODELS/isyn/VAE/dnn_16_cgcnn_test'
os.makedirs(path, exist_ok=True)
writer = SummaryWriter(path)
for i in range(50):
    train_loss = tr.train(train_dl)
    writer.add_scalar('Loss/Train', train_loss, i+1)
    valid_loss, valid_output = tr.test(valid_dl)
    test_loss, test_output = tr.test(test_dl)

    writer.add_scalar('Loss/Valid', valid_loss, i+1)
    writer.add_scalar('Loss/Test', test_loss, i+1)
    writer.add_scalar('KLD/Valid', valid_output['kld'], i+1)
    writer.add_scalar('KLD/Test', test_output['kld'], i+1)


In [4]:
from src.feature import feature_to_composit, ligand_label
def feature_to_ligand_index(feat_vec, tol=0.5):
    comps = feature_to_composit(feat_vec, tol)
    out = []
    for comp in comps:
        eles = '-'.join(comp.keys())
        i = ligand_label.get(eles)
        out.append(-1 if i is None else i)
    return out

inp = feature_to_ligand_index(valid_output['input'])
prd = feature_to_ligand_index(valid_output['pred'])

In [11]:
from sklearn.metrics import confusion_matrix


array([[   0,    0,    0, ...,    0,    0,    0],
       [   0, 8313,  120, ...,    0,    0,    0],
       [   0,  230, 2259, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]])

In [52]:

def _feature_to_composit(feat_vec, tol=0.5, to_string=False, to_idx=False):
    n_feat = feat_vec.shape[-1]
    if n_feat not in [97, 87, 12]:
        raise TypeError(f'feature type is not supported', composit_fnc)
    if n_feat == 97: # active_composit
        ref = ActiveElements
    elif n_feat == 87: # metal_composit
        ref = ['None'] + MetalElements
    elif n_feat == 12: # ligand_composit
        ref = ['Metal'] + LigandElements
    mask = feat_vec > tol
    out = [tuple([ref[i] for i in np.where(vec)[0]]) for vec in mask]
    if to_string or to_idx:
        out = ['-'.join(o) for o in out]
    if to_idx:
        out = [ligand_label[o] if o in ligand_label.keys() else -1 for o in out]
    return out


In [70]:
from sklearn.metrics import f1_score

f1_score(inp, prd, average='micro')

0.9063270336894002

In [72]:
'.'.join(ligand_label.keys())

'Metal-O.Metal-C-O.Metal.Metal-H-O.Metal-N-O.Se.H-N-O-P.S.Metal-H-C-O.Metal-F.Metal-S.Metal-N.Metal-Cl.Metal-H-N-O.Metal-O-P.Metal-H-O-P.P.Metal-Se.C.O-P.Metal-I.Metal-O-S.H-O-P.Metal-Br.Metal-C.Metal-H.H-C-O.H-N-Cl.P-S.Metal-H-N.P-Se.Metal-P.H-N-F.Metal-O-Cl.O-Se.I.H-N-O-S.Metal-C-N.H-O-S.Metal-H-N-O-P.H-N-Br.H-C-S-I.Metal-O-Br.H-C-S-Cl.H-C-S-Br.Metal-C-S.Metal-O-Se.Metal-H-N-Cl.Metal-P-S-Se.H-C-N-S.Metal-O-F-P.N-P.Cl.Metal-P-S.Metal-H-N-O-S.H-Cl.Metal-H-O-S.H-C.Metal-H-C-N-O-P.H-C-N-O.Metal-H-F.Br.Metal-O-F.H-N-O.H-O-Se.Metal-H-Cl.H-S.C-N-S.Metal-O-I.H-C-N.Metal-O-P-Cl.Metal-H-O-Cl.Metal-P-S-Br.H-C-N-Cl.Metal-H-C-N-Cl.H-C-N-I.Metal-H-C-N-I.Metal-H-C-N-Br.Metal-S-Se'

In [18]:
import numpy as np
import matplotlib.pyplot as plt

def cyclical_kld_annealing(epochs, start=0, stop=1, n_cycle=4, ratio=0.5):
    '''
    Code from paper 'Cyclical Annealing Schedule: A Simple Approach to Mitigating KL Vanishing'
    arXiv: https://arxiv.org/abs/1903.10145
    github: https://github.com/haofuml/cyclical_annealing

    Scheduling KLD for better training of VAE.
    '''
    beta = np.ones(epochs)
    period = epochs / n_cycle
    step = (stop - start) / (period * ratio)
    for c in range(n_cycle):
        v, i = start, 0
        while v <= stop and (int(i+c*period) < epochs):
            beta[int(i+c*period)] = v
            v += step
            i += 1
    return beta

def fnc(epochs, start=0, stop=1, n_cycle=4, ratio=0.5):
    beta = np.ones(epochs)
    period = epochs / n_cycle
    step = (stop - start) / (period * ratio)
    for i in range(int(period)):
        beta[i::int(period)] = start + step * i
    return np.clip(beta, start, stop)

epochs = 50000
%timeit fnc(epochs)
%timeit cyclical_kld_annealing(epochs)
#f, axs = plt.subplots(3,1, figsize=(10,8), sharex=True)
#axs[0].scatter(range(epochs), y1)
#axs[1].scatter(range(epochs), y2)
#axs[2].scatter(range(epochs), (np.abs(y1 - y2) > 1e-15).astype(float))


2.98 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.9 ms ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
import sys
sys.path.append('..')
from src.data import ReactionDataset
from src.feature import init_info

init_info(0)
rd = ReactionDataset(precursor_feat_type='active_composit')
rd.from_file()
len(rd)

31387

In [3]:
rd.to('cpu')
feat, info = rd.cfn(rd)

In [4]:
import numpy as np
np.unique(feat['label'].numpy(), return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

# -